In [2]:
import pandas as pd
import numpy as np
import praw

## Connect to Reddit

In [4]:
reddit = praw.Reddit(client_id = '9p5TdvPWFQ2rLQ',
                     client_secret = 'dOvpA6FJO7UwkzbK6hRpbkX08gA',
                     user_agent = 'test')

## From PRAW Tutorial

#### https://praw.readthedocs.io/en/latest/tutorials/comments.html

### Obtain a Submission Object

In [7]:
# Submission URL:
# https://www.reddit.com/r/funny/comments/3g1jfi/buttons/

# With URL
submission = reddit.submission(url='https://www.reddit.com/r/funny/comments/3g1jfi/buttons/')

# With comment ID
#submission = reddit.submission(id='3g1jfi')

#### This is a useful thing to remember in general. (Copy to a reference notebook?)

In [57]:
def method_list(obj):
    return [method for method in dir(obj) if callable(getattr(obj, method)) and not method.startswith("__")]

In [60]:
method_list_submission2 = method_list(submission)

True

In [ ]:
# With a submission object we can then interact with its CommentForest through the submission’s comments attribute.
# A CommentForest is a list of top-level comments each of which contains a CommentForest of replies.

In [55]:
# Not sure how this filters comments
# Does top-down, but does that include children or just parent nodes?
# What are the tie-breakers?
for top_level_comment in submission.comments:
    print(top_level_comment.body)

### Exploring Submission Object

In [50]:
print(submission.created)

#author is the OP username
#likes returned None
#name is t3_3gljfi (3gljfi is the post id)
#num_comments is 775
#num_reports is None
#score is 36875
#title is Buttons
#view_count is None
#visited is False


1438918826.0


### CommentForest Object has a method replace_more()

In [30]:
%%time

sub_comments = []
submission.comments.replace_more(limit=0)

for top_level_comment in submission.comments:
#    print(top_level_comment.body)
    sub_comments.append(top_level_comment)


CPU times: user 1.11 ms, sys: 18 µs, total: 1.13 ms
Wall time: 1.14 ms


### 

In [35]:
submission.comments.replace_more(limit=0)
comment_test = submission.comments.list()[0]



In [ ]:
submission.comments.replace_more(limit=0)
for comment in submission.comments.list():
    print(comment.body)
    #author, depth, score
    
    print(comment.time)
    

In [ ]:
# from CommentForest documentation, toward bottom
# while True:
#     try:
#         submission.comments.replace_more()
#         break
#     except PossibleExceptions:
#         print('Handling replace_more exception')
#         sleep(1)

## Try with a Submission on Subreddit 'Datascience'

In [ ]:
#'https://www.reddit.com/r/datascience/comments/6bo3mk/xkcd_machine_learning/dho6cmf/'

## AB

In [ ]:
# May expand to subreddits other than "datascience", but let's stick to that one for now.
# If and when expansion occurrs, generate a list and iterate over it or something like that.
subreddit = reddit.subreddit('datascience')

### Some reference timestamps

In [ ]:
# http://www.unixtimestamp.com/index.php
1451606400 #2016-01-01, 00:00:00
1483228800 #2017-01-01, 00:00:00

(1483228800 - 1451606400)/60/60/24

## Let's Play Around with the Subreddit Object

#### (What is the relationship between the objects Subreddit and Submission?)

In [71]:
def create_timestamp(start, days):
    daysecs = 24*60*60
    end = start + daysecs*days
    
    return 'timestamp:{}..{}'.format(int(start), int(end))

In [ ]:
def calc_end(start, days):
    daysecs = 24*60*60
    return start + daysecs*days

In [ ]:
def get_stuff(subreddit, start, days, loops):

    comments = []
    post_info = []

    t_stamp = create_timestamp(start, days)

    #sub = reddit.subreddit(subreddit).search(t_stamp, syntax = 'cloudsearch', limit = None)
    sub = reddit.subreddit(subreddit).submissions(start, calc_end(start, days))

    for post in sub:

        pi = {}
        pi['title'] = post.title
        pi['score']= post.score
        pi['id'] = post.id
        pi['selftext'] = post.selftext
        pi['created'] = post.created
        pi['name'] = str(post.author)

        cs = {}
        cs['id'] = post.id

        post.comments.replace_more(limit = 0)

        for i, comment in enumerate(post.comments.list()): #z, y
            name = 'comment{}'.format(i)
            cs[name] = comment.body

        post_info.append(pi)
        comments.append(cs)

        if i >= 10:
            break
        suicide_comments.insert_one(cs)
        suicide_posts.insert_one(pi)

    return comments, post_info

In [ ]:
for num_days in range(0.25, 1.25, 0.25):
    sub = (reddit.subreddit('datascience')
           .search(create_timestamp(1483228800, numdays), syntax = 'cloudsearch', limit = None))
    print(len(list(sub)))